
# PFL0090 - TRABALHO DE REDES NEURAIS ARTIFICIAIS PROFUNDAS

Alunos: 
* ARTUR ANTONIO DAL PRÁ
* FILIPE DA SILVA

## Indice

1. [Objetivo](#1)
2. [Abordagens comuns e trabalhos correlatos](#2)
3. [Proposta implementada](#3)
4. [Resultados alcançados](#4)
5. [Discussão final](#5)
6. [Referências](#6)

## Objetivo <a class="anchor" id="1"></a>

Temos como meta, conseguir segmementar terrenos, visando ter noção de propriedades rurais podendo ter noção de valor, atraves de images de satalete, em 5 grupos, que são:
* Construções; 
* Campos; 
* Florestas;
* Estradas;
* Extenções de Água;

## Abordagens comuns e trabalhos correlatos <a class="anchor" id="2"></a>

Para esse tipo de trabalho, quando executado de forma manual, é necessário ter uma pessoa que analisava as imagens e as marcava manualmente, realizando medições e segmentando os terrenos.

Utilizaríamos o **método de Otsu**, que é um dos algoritmos de limiarização mais populares, utilizado para encontrar um threshold ideal para separar os elementos na frente e no fundo de uma imagem.
Adaptando essa técnica à nossa realidade, imaginamos uma solução para esse problema utilizando técnicas computacionais. Primeiro, aplicaríamos um filtro de cores para evidenciar o limiar e, em seguida, um filtro de borda para obter uma transição nítida entre os elementos desejados. 

![Exemplo de campo por diferenca de cores](img/img_d_02.png)

Após essas etapas, comprimiríamos as imagens e as aplicaríamos em uma rede neural clássica, considerando que cada pixel seria uma entrada da rede.

## Proposta implementada e Resultados alcançados<a class="anchor" id="3"></a>

Das tecnicas estudadas, optamos pela [YOLO8](https://yolov8.com/) que em seu site é definica como "Um novo estado da arte em visão computacional, suportando tarefas de detecção, classificação e segmentação de objetos". Tendo em mente que o mesmo possui uma API de facil configuração e ser open source sendo isso fatores decisivos para a escolha.
Para os dados, usamos uma base disponivel no site do [Roboflow](https://universe.roboflow.com/myproject-gccot/landscape-segmentation/dataset/5/download). Esse dataset possui 2696 imagens para treinamento e 289 para validação.

![logo yolo8](img/img_d_03.png)  ![logo roboflow](img/img_d_04.png) 

Para isso decidimos fazer 3 treinamentos, que serão apresentados a seguir.



In [7]:
from ultralytics import YOLO, settings

settings.update({'runs_dir': './runs'})

model = YOLO('yolov8n-seg')  # yolov8n_custom.yaml

### Treinamento 1

Foi criada uma rede com o treinamento para 30 epocas, sem nenhuma opçao de otmizacao.

In [ ]:
model.train(data='data.yaml',
            epochs=30,
            imgsz=640
            )

![Saida do treinamento da primeira rede.](img/img_a_01.png)

Como podemos ver a saida do treinamento da primeira rede, levou mais de 5 horas.

O Treinamento gerou o seguintes graficos:

![Saida do treinamento da primeira rede.](img/img_a_02.png)

Onde podemos observar as epocas por treinamento.

![Saida do treinamento da primeira rede.](img/img_a_03.png)

Aqui temos a matriz de confusão normalizada.

E por fim alguns exemplos da segmentação:

![Saida do treinamento da primeira rede.](img/img_a_04.jpg)

![Saida do treinamento da primeira rede.](img/img_a_05.jpg)

![Saida do treinamento da primeira rede.](img/img_a_06.jpg)

### Treinamento 2

Para o caso 2 foi criada uma rede com o treinamento para 100 epocas, sem nenhuma opçao de otmizacao.

In [ ]:

model.train(data='data.yaml',
            epochs=100,
            imgsz=640
            )


![Saida do treinamento da segunda rede.](img/img_b_01.png)

Como podemos ver a saida do treinamento da segunda rede, levou mais de 17 horas.

O Treinamento gerou o seguintes graficos:

![Saida do treinamento da segunda rede.](img/img_b_02.png)

Onde podemos observar as epocas por treinamento.

![Saida do treinamento da segunda rede.](img/img_b_03.png)

Aqui temos a matriz de confusão normalizada.

E por fim alguns exemplos da segmentação:

![Saida do treinamento da segunda rede.](img/img_b_04.jpg)

![Saida do treinamento da segunda rede.](img/img_b_05.jpg)

![Saida do treinamento da segunda rede.](img/img_b_06.jpg)

### Treinamento 3

E para o ultimo caso utilizou-se uma rede com 30 epocas, com fatores de otimizacao, sendo o de "patience" o que indica o fim quando existe pouco avanço dentro do treinamento de epocas.

In [ ]:

model.train(data='data.yaml',
            epochs=30,
            patience=8,
            batch=16,  # number of images per batch (-1 for AutoBatch)
            imgsz=640,
            workers=8,
            pretrained=True,
            resume=False,  # resume training from last checkpoint
            single_cls=False,  # Whether all classes will be the same (just one class)
            # project='runs/detect',  # Default = /home/{user}/Documents/ultralytics/runs
            box=7.5,  # More recall, better IoU, less precission, 
            cls=0.5,  # Bbox class better
            dfl=1.5,  # Distribution Focal Loss. Better bbox boundaries
            val=True,
            # Augmentations
            degrees=0.3,
            hsv_s=0.3,
            hsv_v=0.3,
            scale=0.5,
            fliplr=0.5
            )


![Saida do treinamento da terceira rede.](img/img_c_01.png)

Como podemos ver a saida do treinamento da terceira rede, levou mais de 1 hora e 30 minutos.

O Treinamento gerou o seguintes graficos:

![Saida do treinamento da terceira rede.](img/img_c_02.png)

Onde podemos observar as epocas por treinamento.

![Saida do treinamento da terceira rede.](img/img_c_03.png)

Aqui temos a matriz de confusão normalizada.

E por fim alguns exemplos da segmentação:

![Saida do treinamento da terceira rede.](img/img_c_04.jpg)

![Saida do treinamento da terceira rede.](img/img_c_05.jpg)

![Saida do treinamento da terceira rede.](img/img_c_06.jpg)

## Discussão final <a class="anchor" id="5"></a>

Observamos que todos os modelos alcançaram a convergência e estão prontos para serem implementados em produção como uma primeira versão. Para aumentar a precisão dos modelos, poderíamos considerar o uso de uma base de imagens de satélite. Como estamos utilizando o Google Maps nas atividades práticas da disciplina, essa abordagem poderia proporcionar uma acurácia superior. Isso se deve ao fato de que o conjunto de treinamento é definido pela altura de voo do drone, conforme especificado na descrição do dataset, que varia de acordo com o zoom e o controle de altitude.

![Imagem obtida pelo modelo 2 com google maps.](img/img_d_01.png)

Ao analisar a matriz de confusão, notamos que a segmentação referente à área de floresta frequentemente se confunde com o background. Isso é atribuído ao formato das florestas segmentadas no dataset estrangeiro. Na nossa região, a vegetação nativa apresenta afloramentos de rochas e não temos uma continuidade tendo esparçamento em alguns trechos (algumas vezes devido ao desmatamento irregular ou outras questões geologicas), resultando em uma cobertura florestal irregular.

![Matriz de confusao.](img/img_b_03.png)

Para melhorar as previsões no futuro, aprimorar a qualidade das imagens obtidas via satélite, considerando também uma refatoração do dataset, seria benéfico. Isso proporcionaria imagens mais nítidas e detalhadas, beneficiando a identificação de elementos como construções. Atualmente, a falta de nitidez nessas imagens afeta a resolução das previsões, e uma melhoria nesse aspecto resultaria em uma predição mais precisa.


### Exemplo prático

Iremos agora usar um metodo de captura de tela para por a prova os modelos obtidos.

In [8]:
from PIL import ImageGrab
import cv2
import numpy as np
from collections import defaultdict

def real_time_maps(model):

    track_history = defaultdict(lambda: [])

    while True:
        img = ImageGrab.grab(bbox=(100,100,1000,1000))
    
        results = model.track(img, persist=True)
    
        # Process results list
        for result in results:
            # Visualize the results on the frame
            img = result.plot()
    
            if True:
                try:
                    # Get the boxes and track IDs
                    boxes = result.boxes.xywh.cpu()
                    track_ids = result.boxes.id.int().cpu().tolist()
    
                    # Plot the tracks
                    for box, track_id in zip(boxes, track_ids):
                        x, y, w, h = box
                        track = track_history[track_id]
                        track.append((float(x), float(y)))  # x, y center point
                        if len(track) > 30:  # retain 90 tracks for 90 frames
                            track.pop(0)
    
                        # Draw the tracking lines
                        points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                        cv2.polylines(img, [points], isClosed=False, color=(230, 0, 0), thickness=5)
                except:
                    pass
                
        cv2.imshow('Imagem do GoogleMaps', img)
    
        k = cv2.waitKey(1)
        if k == ord('q'):
            break
        
    cv2.destroyAllWindows()

In [ ]:
model = YOLO('runs/segment/train5/weights/best.pt')

real_time_maps(model)

In [ ]:
model = YOLO('runs/segment/train52/weights/best.pt')

real_time_maps(model)

In [ ]:
model = YOLO('runs/segment/train522/weights/best.pt')

real_time_maps(model)

## Referências  <a class="anchor" id="6"></a>

* https://universe.roboflow.com/myproject-gccot/landscape-segmentation/dataset/5/download
* https://github.com/WellingtonDev25/yolov8_amostra/blob/main/YoloV8_Amostra.ipynb
* https://csvisaocomputacional.com.br/cursoyolo
* https://docs.ultralytics.com/modes/track/#multithreaded-tracking
* https://www.kaggle.com/datasets/sshikamaru/car-object-detection/code
* http://profs.ic.uff.br/~aconci/OtsuTexto.pdf
* https://www.youtube.com/watch?v=JVqU9IBZaAg